In [2]:
import numpy as np
import numba
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from plotting import plot

In [3]:
thrust = 210000.0
g0 = 1.61544
STANDARD_GRAV = 9.81
Isp = 265.2 

params = (thrust, g0, Isp)

x0 = np.array([0,0,0,0,50000])

In [4]:
@numba.njit
def dynamics(x, u, params):

    thrust = params[0]
    g0 = params[1]
    Isp = params[2]
    
    cos_theta  = np.cos(u)
    sin_theta  = np.sin(u)
    dx = np.zeros_like(x)
    dx[:,0] = x[:,3]
    dx[:,1] = x[:,4]
    dx[:,2] =  (thrust * cos_theta) / x[:,4] 
    dx[:,3] =  (thrust * sin_theta)  / x[:,4] - g0
    dx[:,4] =  -thrust / STANDARD_GRAV / Isp
    return dx

@numba.njit
def objective(N, dt):
    return N*dt

# Define the dynamics defects
@numba.njit
def state_defects(decision_variables, args):
    N = args[0] 
    states_dim = args[1] 
    params = args[2] 
    # Index [0 N )in the decision_variables vector contains the control inputs 
    u = decision_variables[:N]
    # Index [N -1] in the decision_variables vector contains the state  
    x = decision_variables[N:].reshape((N+1, states_dim))
    dt = decision_variables[N:-1] # Time is the last decision var here
    # Calculate the dynamics
    x_dot = dynamics(x, u)
    # Calculate the approximation of integral using trapezoidal quadrature
    integral = ((x_dot[:-1] + x_dot[1:])) / 2 * dt
    # Calculate the state defects
    defects = x[1:] - x[:-1]  - integral

    return defects.transpose().flatten()

In [5]:
# Define the initial and final states 
x0 = np.array([0,0,0,0,50000])
terminal_x = [None, 1.8500e+05, 1.6270e+03, 0, None]
_, xf_y, xf_vx, xf_vy, _ = terminal_x

states_dim = 5
N = 50 # Number of collocation points, 

#bounds of pitch angle
u_b = np.pi /2 
l_b = -np.pi /2 

In [6]:
mdot = thrust / STANDARD_GRAV / Isp
t_max = 50000 / mdot
dt_max = t_max/ N

In [7]:
# Initial guess for control inputs
u_init = np.linspace(1.5, 0.76, N) # Decision variable
t_init  = np.linspace(0, t_max, N+1) 
dt_init = t_init[1] - t_init[0]  # time step
params =(N,states_dim)

# Initial guess for states
x_init = np.zeros((N+1, states_dim))
x_init[:, 0] = np.linspace(x0[0], 215e3, N+1) 
x_init[:, 1] = np.linspace(0, xf_y, N+1)
x_init[:, 2] = np.linspace(0, xf_vx, N+1)
x_init[:, 3] = np.linspace(0, xf_vy, N+1)
x_init[:, 4] = np.linspace(50000, 0.1 *  50000, N+1)

# Concatenate control inputs and states into a single decision variable
initial_guess = np.concatenate([u_init, x_init.flatten(), [12]])


In [14]:
## Plot initial Guesses 

# plot( t_init, [x_init[:, 0], x_init[:, 1]])
# plot( t_init[:-1], u_init)

In [10]:

# Define the optimization problem
def problem(decision_variables, args):
    N = args[0] 
    dt = decision_variables[-1]

    obj_value = objective(N, dt)
    
    return obj_value

# Define the bounds for the decision variables
bounds = [(l_b, l_b)] * N # this case it is the control parameter for the trajectory, the pitch command
num_state_bounds = states_dim*(N+1)

state_bounds = [(0, None)] * num_state_bounds

# for i in range(0,N+1):
#     state_bounds[states_dim*i] = (0,None)

xf_y, xf_vx, xf_vy
bounds = bounds + state_bounds    
#Enforcing Bound constraint on initial and final states
bounds[N]    = (0.0,0.0) # x 
bounds[N+1]  = (0.0,0.0) # y
bounds[N+2]  = (0.0, 0.0) # v_x 
bounds[N+3] = (0.0, 0.0) # v_y 
bounds[N+4] = (50000, 50000) # mass

# Final Bounds
bounds[N+ num_state_bounds - states_dim + 0] = (0.0,None) # x 
bounds[N+ num_state_bounds - states_dim + 1] = (xf_y,xf_y) # y
bounds[N+ num_state_bounds - states_dim + 2] = (xf_vx, xf_vx) # v_x 
bounds[N+ num_state_bounds - states_dim + 3] = (xf_vy, xf_vy) # v_y 
bounds[N+ num_state_bounds - states_dim + 4] = (0, 30000) # mass
bounds = bounds + [(5, 12.3)] # Adding time bound 
arguments = (params,)
# Define the constraints
constraints = [{'type': 'eq', 'fun': state_defects, 'args':arguments },]


In [11]:
problem(initial_guess, arguments)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function mul>) found for signature:
 
 >>> mul(UniTuple(int64 x 2), float64)
 
There are 14 candidate implementations:
  - Of which 12 did not match due to:
  Overload of function 'mul': File: <numerous>: Line N/A.
    With argument(s): '(UniTuple(int64 x 2), float64)':
   No match.
  - Of which 2 did not match due to:
  Operator Overload in function 'mul': File: unknown: Line unknown.
    With argument(s): '(UniTuple(int64 x 2), float64)':
   No match for registered cases:
    * (int64, int64) -> int64
    * (int64, uint64) -> int64
    * (uint64, int64) -> int64
    * (uint64, uint64) -> uint64
    * (float32, float32) -> float32
    * (float64, float64) -> float64
    * (complex64, complex64) -> complex64
    * (complex128, complex128) -> complex128

During: typing of intrinsic-call at /tmp/ipykernel_3989/3658962887.py (20)

File "../../../../tmp/ipykernel_3989/36

In [ ]:
# Solve the optimization problem

result = minimize(problem, initial_guess, method='SLSQP', bounds=bounds, args=arguments, constraints=constraints)

In [ ]:
# Extract the optimal control inputs and states
u_opt = result.x[:N]

x_opt = result.x[N:].reshape((N+1, states_dim))